In [1]:
import pandas as pd
from pam.read import read_matsim
from pam.planner import choice
from pam.operations.cropping import link_population
from pam.planner.od import OD, Labels
import numpy as np
import os

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
population = read_matsim(os.path.join('..', 'tests', 'test_data', 'test_matsim_plansv12.xml'))
link_population(population)
for hid, pid, person in population.people():
    for act in person.activities:
        act.location.area = 'a' 

In [4]:
data_zones = pd.DataFrame(
    {
        'zone': ['a', 'b'],
        'jobs': [100, 200],
        'schools': [3, 1]
    }
).set_index('zone')
data_zones

,jobs,schools
zone,,
a,100,3
b,200,1


In [5]:
data_od = np.array(
    [
        [
            [[20, 40], [40, 20]],
            [[5, 8], [8, 5]]
        ],
        [
            [[30, 45], [45, 30]],
            [[5, 8], [8, 5]]
        ]
    ]
)

labels = {
    'mode': ['car', 'bus'],
    'vars': ['time', 'distance'],
    'origin_zones': ['a', 'b'],
    'destination_zones': ['a', 'b']
}


od = OD(
    data=data_od,
    labels=labels
)

od.data

array([[[[20, 40],
         [40, 20]],

        [[ 5,  8],
         [ 8,  5]]],


       [[[30, 45],
         [45, 30]],

        [[ 5,  8],
         [ 8,  5]]]])

In [6]:
planner = choice.ChoiceMNL(population, od, data_zones)

In [8]:
# get the choice set, home-based work
scope_act = 'work'
var = 'time'
asc = 1
beta_time = [1, 2] # one beta for each mode

u = """ \
    asc + (1 * (person.attributes['subpopulation']=='poor')) * \
    (zones['jobs'].values / (beta_time * od[:, 'time', person.home.area]))
"""
index_set = []
choice_set = []
od = planner.od
zones = planner.zones
for hid, hh in planner.population:
    for pid, person in hh:
        for i, act in enumerate(person.activities):
            if act.act == scope_act: #TODO: scope of function + mandatory locations
                c = {
                    'pid': pid,
                    'hid': hid,
                    'seq': i,
                    'home_loc': person.home.area,
                    'act_loc': act.location.area,
                    'act': act
                }
                p = eval(u)
                p = p.flatten()
                choice_set.append(p)
                index_set.append(c)
        
index_set = pd.DataFrame(index_set)
choice_set = np.array(choice_set)
print(index_set)
choice_set

      pid     hid  seq home_loc act_loc   
0   chris   chris    1        a       a  \
1  fatema  fatema    1        a       a   
2    fred    fred    3        a       a   
3   gerry   gerry    3        a       a   
4    nick    nick    1        a       a   

                                                 act  
0  Activity(act:work, location:POINT (10100 0), t...  
1  Activity(act:work, location:POINT (10100 0), t...  
2  Activity(act:work, location:POINT (10100 0), t...  
3  Activity(act:work, location:POINT (10100 0), t...  
4  Activity(act:work, location:POINT (0 10000), t...  


array([[1.        , 1.        , 1.        , 1.        ],
       [6.        , 3.5       , 4.33333333, 3.22222222],
       [6.        , 3.5       , 4.33333333, 3.22222222],
       [6.        , 3.5       , 4.33333333, 3.22222222],
       [1.        , 1.        , 1.        , 1.        ]])